In [10]:
#sudo pip install ibm_db
import ibm_db
# conda install -c conda-forge ibm_db

import pandas as pd
import numpy as np
import psycopg2
# conda install -c anaconda psycopg2
import sqlalchemy
# conda install -c anaconda sqlalchemy
import matplotlib as plt
import requests
# conda install -c anaconda requests
from requests.auth import HTTPDigestAuth
import json
# conda install -c jmcmurray json
import urllib.request
import urllib.parse
# conda install -c anaconda zlib
import sys
import asyncio

from sqlalchemy import create_engine

import ibm_db_sa
# conda install -c auto ibm_db_sa
# pip install ibm-db-sa

import ibm_db_dbi

import logging

import datetime

from my_functions import *

In [11]:
logging.basicConfig(filename='logs.log',level=logging.DEBUG)


POSTGRES_ADDRESS = 'localhost'
POSTGRES_PORT = '5432'
POSTGRES_USERNAME = 'ebreshna_user'
POSTGRES_PASSWORD = 'secret'
POSTGRES_DBNAME = 'ebreshna_test'
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'.format(username=POSTGRES_USERNAME, password=POSTGRES_PASSWORD, ipaddress=POSTGRES_ADDRESS, port=POSTGRES_PORT, dbname=POSTGRES_DBNAME))
# conn = psycopg2.connect("host=localhost dbname=asims_test user=asims_user password=secret port=5432")
# cur = conn.cursor()
psql_engine = create_engine(postgres_str)

# SQL SERVER CONNECTION
DB2_ADDRESS = '10.10.100.47'
DB2_PORT = '53000'
DB2_USERNAME = 'db2inst1'
DB2_PASSWORD = 'db2inst1'
DB2_DBNAME = 'mpower'
# db2_engine = create_engine('ibm_db_sa+pyodbc:///?odbc_connect={}'.format(quoted))
# db2_engine = create_engine("db2+ibm_db://user:pass@10.10.100.47:51000/mpower")
# conn = ibm_db.connect("DATABASE=mpower;HOSTNAME=host;PORT=50000;PROTOCOL=TCPIP;UID=username;PWD=password;", "", "")
db2_conn = ibm_db.connect("DRIVER={{IBM DB2 ODBC DRIVER}};DATABASE=mpower;HOSTNAME=10.10.100.47;PORT=53000;PROTOCOL=TCPIP;UID=db2inst1;PWD=db2inst1;", "", "")
# this is required to enable database access through Pandas
db2_conni =  ibm_db_dbi.connect(u"DRIVER={{IBM DB2 ODBC DRIVER}};DATABASE=mpower;HOSTNAME=10.10.100.47;PORT=53000;PROTOCOL=TCPIP;UID=db2inst1;PWD=db2inst1;")

In [12]:
# Region
cols_map = [
    {'src_col': 'ID', 'des_col': 'id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'PHASE_LINE_IN_AREA', 'des_col': 'phase_line_in_area', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'TYPE_OF_WIRE', 'des_col': 'type_of_wire', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'VOLTAGE', 'des_col': 'voltage', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'H_DISTANCE', 'des_col': 'h_distance', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'V_DISTANCE', 'des_col': 'v_distance', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'IS_METER_AVAIL', 'des_col': 'is_meter_avail', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'NO_METERS', 'des_col': 'no_meters', 'src_dtype': 'DECIMAL', 'des_dtype': 'smallint'},
    {'src_col': 'MTR_CONS_NO', 'des_col': 'mtr_cons_no', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'IS_LINE_DISCONNECTED', 'des_col': 'is_line_disconnected', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'DISCONN_CONS_NO', 'des_col': 'disconn_cons_no', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'IS_DUE_BAL', 'des_col': 'is_due_bal', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'IS_CABLE_PASS_NEIG', 'des_col': 'is_cable_pass_neig', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'IS_SERV_CABLE_PASS_NEIG', 'des_col': 'is_serv_cable_pass_neig', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'IS_STAY_LINE_PASS_NEIG', 'des_col': 'is_stay_line_pass_neig', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'IS_SHUTDOWN_IMMEDIATELY', 'des_col': 'is_shutdown_immediately', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'IS_SERV_CABLE_PASS_VOLT', 'des_col': 'is_serv_cable_pass_volt', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'IS_ANY_HIGH_MED_VOLT_LINES', 'des_col': 'is_any_high_med_volt_lines', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'HIGH_MED_LINE_VOLTAGE', 'des_col': 'high_med_line_voltage', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'H_DIST_H_M_LINE', 'des_col': 'h_dist_h_m_line', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'V_DIST_H_M_LINE', 'des_col': 'v_dist_h_m_line', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'DESCRIPTION_OF_HOUSE', 'des_col': 'description_of_house', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'IS_ROLE_REQUIRED', 'des_col': 'is_role_required', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'ROLE_NUM', 'des_col': 'role_num', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'IS_STAY_REQUIRED', 'des_col': 'is_stay_required', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'STAY_NUM', 'des_col': 'stay_num', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'IS_BRAC_JUN_BOX_REQU', 'des_col': 'is_brac_jun_box_requ', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'BRAC_JUN_BOX_NUM', 'des_col': 'brac_jun_box_num', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'PLACE_OF_CONN', 'des_col': 'place_of_conn', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'DTR_ID', 'des_col': 'dtr_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'DTR_CAPACITY', 'des_col': 'dtr_capacity', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'PLACE_OF_METER_INSTAL', 'des_col': 'place_of_meter_instal', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'TYPE_OF_WIRE_TO_CONNECT', 'des_col': 'type_of_wire_to_connect', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'PLACE_OF_WALL_BRACKET', 'des_col': 'place_of_wall_bracket', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'CREATED_BY', 'des_col': 'created_by', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'CREATE_DATE', 'des_col': 'create_date', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'TIMESTAMP'},
    {'src_col': 'UPDATED_BY', 'des_col': 'updated_by', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'UPDATE_DATE', 'des_col': 'update_date', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'TIMESTAMP'},
    {'src_col': 'DB_IP_ADDRESS', 'des_col': 'db_ip_address', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'SOURCE_OFFICE_ID', 'des_col': 'source_office_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'TARGET_OFFICE_ID', 'des_col': 'target_office_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'RECORD_STATUS', 'des_col': 'record_status', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'SCREEN_ID', 'des_col': 'screen_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'MONTH', 'des_col': 'month', 'src_dtype': 'DECFLOAT', 'des_dtype': 'String'},
    {'src_col': 'YEAR', 'des_col': 'year', 'src_dtype': 'DECFLOAT', 'des_dtype': 'String'},
    {'src_col': 'FLAG1', 'des_col': 'flag1', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'FLAG2', 'des_col': 'flag2', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'FLAG3', 'des_col': 'flag3', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'FLAG4', 'des_col': 'flag4', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'FLAG5', 'des_col': 'flag5', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'MODULE_ID', 'des_col': 'module_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'SUBMODULE_ID', 'des_col': 'submodule_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'REGISTRATION_ID', 'des_col': 'registration_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'SHUTDOWN_CHARGES', 'des_col': 'shutdown_charges', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'NAMEOFINSPECTOR', 'des_col': 'nameofinspector', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'DESIGINSPECTOR', 'des_col': 'desiginspector', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'INSPECTED_ON', 'des_col': 'inspected_on', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'TIMESTAMP'},
    {'src_col': 'CHECK_CONDITION', 'des_col': 'check_condition', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'IS_HOUSE_OLD', 'des_col': 'is_house_old', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'CONDITION_OF_WIRING', 'des_col': 'condition_of_wiring', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'NO_FLOORS', 'des_col': 'no_floors', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'IS_POLE_REQUIRED', 'des_col': 'is_pole_required', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'NO_OF_POLES', 'des_col': 'no_of_poles', 'src_dtype': 'DECIMAL', 'des_dtype': 'smallint'},
    {'src_col': 'DUE_BAL_CONS_NUM', 'des_col': 'due_bal_cons_num', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'DUE_BAL', 'des_col': 'due_bal', 'src_dtype': 'DECIMAL', 'des_dtype': 'NUMERIC'},
    {'src_col': 'IS_EST_REQUIRED', 'des_col': 'is_est_required', 'src_dtype': 'CHAR', 'des_dtype': 'Character'},
    {'src_col': 'INSP_REMARKS', 'des_col': 'insp_remarks', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'LOCATION', 'des_col': 'location', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'SUB_LOCATION', 'des_col': 'sub_location', 'src_dtype': 'String', 'des_dtype': 'String'}
]

migrated_row_count = migrate_data(db2_conni, psql_engine, 'DABS    .NCTINSPECTION_REPORT_T', 'public', 'nctinspection_report_t', cols_map)
print("Total Migrated Records (nctinspection_report_t): " + str(migrated_row_count))


Total Migrated Records (nctinspection_report_t): Total Src Records: 8 Total migrated Records to dest: 16
